In [ ]:
#| default_exp datasets.tid2008

# TID2008

> Building a `tf.data.Dataset` for TID2008.

In [ ]:
#| hide
import os; os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#| export
from pathlib import Path
from typing import List

import pandas as pd
import tensorflow as tf
import cv2

After setting up the path to the directory and loading the corresponding `.csv` file, we need to create a generator that will iterate over the dataframe, load and return a 3-tuple: `(Reference Image, Distorted Image, DMOS)`. Before, we were using a generator to fetch the data but by creating the dataset with only the paths to the images and their mos and mapping the function used to load them, we can obtain massive speed-ups due to the paralelization capabilities being increased.

In [ ]:
#| export
class TID2008():
    """Builder for the TID2008 dataset"""

    def __init__(self,
                 path, # Path to the root directory of the dataset.
                 exclude_imgs: List[int] = None, # Image ID's to exclude.
                 exclude_dist: List[int] = None, # Distortion ID's to exclude.
                 exclude_ints: List[int] = None, # Distortion Intensities ID's to exclude.
                 num_parallel_calls: int = tf.data.AUTOTUNE, # Number of parallel calls when loading the images.
                 ):
        self.path_root = Path(path) if isinstance(path, str) else path
        self.path_ref = self.path_root/"reference_images"
        self.path_dist = self.path_root/"distorted_images"
        self.path_csv = self.path_root/"image_pairs_mos.csv"
        self.data = self.load_data(self.path_csv, exclude_imgs, exclude_dist, exclude_ints)
        self.paths_ref = [str(self.path_ref/p) for p in self.data["Reference"]]
        self.paths_dist = [str(self.path_dist/p) for p in self.data["Distorted"]]
        self.num_parallel_calls = num_parallel_calls
        # self.paths_pair_mos = tf.data.Dataset.from_tensor_slices((self.paths_ref, self.paths_dist, self.data["MOS"]))

    @property
    def dataset(self):
        """tf.data.Dataset object built from the TID2008 dataset."""
        return tf.data.Dataset.from_tensor_slices((self.paths_ref, self.paths_dist, self.data["MOS"]))\
                              .map(self.preprocess, num_parallel_calls=self.num_parallel_calls)

    @staticmethod
    def preprocess(path_ref,
                   path_dist,
                   mos,
                   ):
        img_ref = tf.io.read_file(path_ref)
        img_dist = tf.io.read_file(path_dist)

        img_ref = tf.image.decode_bmp(img_ref, channels=3)
        img_dist = tf.image.decode_bmp(img_dist, channels=3)

        img_ref = tf.image.convert_image_dtype(img_ref, dtype=tf.float32)
        img_dist = tf.image.convert_image_dtype(img_dist, dtype=tf.float32)

        return img_ref, img_dist, mos

    def data_gen(self):
        """Dataset generator to build the tf.data.Dataset."""
        for i, row in self.data.iterrows():
            ref, dist, mos = row.Reference, row.Distorted, row.MOS
            dist = cv2.imread(str(self.path_dist/dist))
            dist = cv2.cvtColor(dist, cv2.COLOR_BGR2RGB)/255.0
            ref = cv2.imread(str(self.path_ref/ref))
            ref = cv2.cvtColor(ref, cv2.COLOR_BGR2RGB)/255.0
            yield ref, dist, mos

    def load_data(self,
                  path,
                  exclude_imgs,
                  exclude_dist,
                  exclude_ints,
                  ):
        data = pd.read_csv(self.path_csv, index_col=0)
        data = data[~data.Reference_ID.isin(exclude_imgs)] if exclude_imgs is not None else data
        data = data[~data.Reference_ID.isin(exclude_dist)] if exclude_dist is not None else data
        data = data[~data.Reference_ID.isin(exclude_ints)] if exclude_ints is not None else data
        return data


In [ ]:
# l = TID2008(path = Path("/media/disk/databases/BBDD_video_image/Image_Quality/TID/TID2008"))
l = TID2008(path = Path("/lustre/ific.uv.es/ml/uv075/Databases/IQA/TID/TID2008/"))

In [ ]:
for a, b, c in l.dataset:
    break
assert a.shape == b.shape

2023-09-13 14:52:07.981337: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-13 14:52:07.981424: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: mlui02.ific.uv.es
2023-09-13 14:52:07.981448: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: mlui02.ific.uv.es
2023-09-13 14:52:07.981611: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.54.3
2023-09-13 14:52:07.981683: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.54.3
2023-09-13 14:52:07.981698: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.54.3
2023-09-13 14:52:07.983199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CP

And we can benchmark it to finish:

In [ ]:
#| eval: false
from tqdm.auto import tqdm
for a, b, c in tqdm(l.dataset): pass

  0%|          | 0/1700 [00:00<?, ?it/s]